In [1]:
# Cross Validation with XGBoost

import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder
from sklearn.cross_validation import cross_val_score
from sklearn.cross_validation import KFold
import xgboost

In [2]:
Load only the users with known destination

train_users = pd.read_csv('../cache/train_users.csv')

In [3]:
Replace NaN values with -1. 

train_users.fillna(-1, inplace=True)

Select proper X and y

y_train = train_users['country_destination']
train_users.drop(['country_destination', 'id'], axis=1, inplace=True)
x_train = train_users.values

label_encoder = LabelEncoder()
encoded_y_train = label_encoder.fit_transform(y_train)

In [4]:
To use `xgboost` models we need a `DMatrix`. This can be done with the next command:

train_data = xgboost.DMatrix(x_train, encoded_y_train)

In [89]:
def customized_eval(preds, dtrain):
    labels = dtrain.get_label()
    top = []

    for i in range(preds.shape[0]):
        top.append(np.argsort(preds[i])[::-1][:5])

    mat = np.reshape(np.repeat(labels,np.shape(top)[1]) == np.array(top).ravel(),np.array(top).shape).astype(int)
    score = np.mean(np.sum(mat/np.log2(np.arange(2, mat.shape[1] + 2)),axis = 1))
    return 'ndcg5', score

array([[0, 2, 1],
       [2, 1, 0],
       [2, 1, 0]])

CPU times: user 4 ms, sys: 0 ns, total: 4 ms
Wall time: 317 µs


('ndcg5', 0.83333333333333337)

In [28]:
param = {
    'max_depth': 1,
    'learning_rate': 1,
    'n_estimators': 5,
    'objective': 'multi:softprob',
    'num_class': 12,
    'gamma': 0,
    'min_child_weight': 1,
    'max_delta_step': 0,
    'subsample': 1,
    'colsample_bytree': 1,
    'colsample_bylevel': 1,
    'reg_alpha': 0,
    'reg_lambda': 1,
    'scale_pos_weight': 1,
    'base_score': 0.5,
    'missing': None,
    'silent': True,
    'nthread': 4,
    'seed': 42
}

num_round = 10
xgboost.cv(param, train_data, num_boost_round=num_round, metrics=['mlogloss'], feval=customized_eval)

KeyboardInterrupt: 